In [15]:
import pandas as pd
import warnings
from utils import KnowYourData

warnings.filterwarnings('ignore')

In [16]:
df = pd.read_excel("db/dataset_v1.xlsx")
df.head(2)

,Type,subject,body,opened,meeting link clicked,responded,meeting_link_clicked
0,example1,🚀 Propel Your Marketing ROI with Advanced Anal...,"Hey [Recipient's Name],\n\nReady to see your m...",0,0.0,0,NaN
1,example1,Data is Your Superpower 📊 Unlock Insights with Us,"Hi [Recipient's Name],\n\nI'm reaching out aga...",1,0.0,0,NaN


### 1. DataSet Overview

In [17]:
data_quality_results = KnowYourData(df = df)
res1 = data_quality_results.analyze_data()
res1

{'data_dimensions': (154, 7), 'features_count': 7, 'sample_count': 154}

In [18]:
res2 = data_quality_results.feature_analysis()
res2

{'numerical_features': ['opened',
  'meeting link clicked',
  'responded',
  'meeting_link_clicked'],
 'count_of_numerical_features': 4,
 'categorical_features': ['Type', 'subject', 'body'],
 'count_of_categorical_features': 3}

In [19]:
res3 = data_quality_results.missing_value_estimation()
res3

{'is_missing_value': 1,
 'sum_of_missing_value': 154,
 'missing_value_feature_wise': '{"Type":0,"subject":0,"body":0,"opened":0,"meeting link clicked":16,"responded":0,"meeting_link_clicked":138}'}

In [20]:
res4 = data_quality_results.data_quality_analysis()
res4

{'is_missing_values': Type                      0
 subject                   0
 body                      0
 opened                    0
 meeting link clicked     16
 responded                 0
 meeting_link_clicked    138
 dtype: int64,
 'is_duplicates': 0}

`Inference`:

- There are 154 samples with 7 fields.
- 3 Categorical and 4 Numerical features.
- Features **meeting link clicked** and **meeting_link_clicked** mean the same. Duplicated Features.
- Presence of Missing values in 2 duplicated columns that when merged together will impute missing values.

### 2. Data Cleaning

2.1.  Imputing Missing values by Merging two duplicated columns, namely - meeting link clicked and meeting_link_clicked

2.2. Many related variables in `Type` column

2.3. Data Type Standarization eg: float to int conversion

In [21]:
# 1. Imputing Missing values by Merging two duplicated columns, namely - meeting link clicked and meeting_link_clicked

df["meeting link clicked"].fillna(0,inplace=True)
df["meeting_link_clicked"].fillna(0,inplace=True)

df["meeting_link_click"] = df["meeting link clicked"] + df["meeting_link_clicked"]
df = df.drop(['meeting link clicked', 'meeting_link_clicked'], axis=1)

df.head(2)

,Type,subject,body,opened,responded,meeting_link_click
0,example1,🚀 Propel Your Marketing ROI with Advanced Anal...,"Hey [Recipient's Name],\n\nReady to see your m...",0,0,0.0
1,example1,Data is Your Superpower 📊 Unlock Insights with Us,"Hi [Recipient's Name],\n\nI'm reaching out aga...",1,0,0.0


In [27]:
# 2. Many related variables in `Type` column

df['Type'] = df['Type'].str.lower().str.strip()

type_mapping = {
    'example1': 'example 1',
    'series1': 'series 1',
    'marketinganalyticsseries': 'marketing analytics series',
    'hrconsultingseries': 'hr consulting series',
    'hr_consulting_series': 'hr consulting series',
    'series_marketing_analytics': 'marketing analytics series',
    'financial_advisory_series': 'financial advisory series',
    'series_itsolutions': 'series it solutions',
    'series1_hr_consulting': 'hr consulting series',
    'email_series_marketing_analytics': 'marketing analytics series',
    'marketinganalytics0': 'marketing analytics series',
    'marketinganalyticsseries': 'marketing analytics series',
    'series_itsolutions': 'series it solutions',
    'legal_services0': 'legal services',
    'it_solutions_series': 'series it solutions',
    'marketing_analytics_series': 'marketing analytics series',
    'series_legal': 'legal services',
    'series_it_solutions':'series it solutions',
}

# Apply the mapping
df['Type'] = df['Type'].replace(type_mapping)


In [28]:
df["Type"].value_counts()

example 1                     81
marketing analytics series    26
hr consulting series          18
series 1                      10
series it solutions            9
legal services                 5
financial advisory series      5
Name: Type, dtype: int64

In [24]:
# 3. DataType Standarization

df["opened"] = df["opened"].astype(int)
df["responded"] = df["responded"].astype(int)
df["meeting_link_click"] = df["meeting_link_click"].astype(int)

### 3. Feature Analysis

In [30]:
df.head(2)

,Type,subject,body,opened,responded,meeting_link_click
0,example 1,🚀 Propel Your Marketing ROI with Advanced Anal...,"Hey [Recipient's Name],\n\nReady to see your m...",0,0,0
1,example 1,Data is Your Superpower 📊 Unlock Insights with Us,"Hi [Recipient's Name],\n\nI'm reaching out aga...",1,0,0


1. `Opened` Univariate Feature Analysis

In [31]:
import plotly.express as px

# Assuming df is your DataFrame
opened_counts = df["opened"].value_counts().reset_index()
opened_counts.columns = ["opened", "count"]

# Plotly Bar Chart
fig = px.bar(opened_counts, x="opened", y="count", color="opened",
             labels={"opened": "Opened", "count": "Count"},
             title="Count of Emails Opened",
             color_discrete_map={"TRUE": "green", "FALSE": "red"})

# Adjust the width
fig.update_layout(width=400)  # Set your desired width

# Use a custom color scale with only 2 colors
fig.update_traces(marker_color=["green", "red"])

fig.show()

2. `Meeting Link Click` Feature Analysis

In [34]:
import plotly.express as px

# Assuming df is your DataFrame
meeting_link_clicked_counts = df["meeting_link_click"].value_counts().reset_index()
meeting_link_clicked_counts.columns = ["meeting_link_click", "count"]

# Plotly Bar Chart
fig = px.bar(meeting_link_clicked_counts, x="meeting_link_click", y="count", color="meeting_link_click",
             labels={"meeting_link_click": "Meeting Link Clicked", "count": "Count"},
             title="Count of Meeting Links Clicked",
             color_discrete_map={"TRUE": "blue", "FALSE": "orange"})

# Adjust the width
fig.update_layout(width=400)  # Set your desired width

# Use a custom color scale with only 2 colors
fig.update_traces(marker_color=["red", "green"])

fig.show()


3. `Responded` Feature Univariate Analysis

In [37]:
import plotly.express as px

# Count the values
responded_counts = df["responded"].value_counts().reset_index()
responded_counts.columns = ["responded", "count"]

# Plotly Bar Chart
fig = px.bar(responded_counts, x="responded", y="count", color="responded",
             labels={"responded": "Responded", "count": "Count"},
             title="Count of Responses",
             color_discrete_map={"TRUE": "blue", "FALSE": "orange"})

# Adjust the width
fig.update_layout(width=600)  # Set your desired width

# Use a custom color scale with only 2 colors
fig.update_traces(marker_color=["red", "green"])

fig.show()


### 4. Conversion Rate - Progression Plot

In [38]:
# Calculate counts for each category
progression_counts = df[["opened", "meeting_link_click", "responded"]].sum().reset_index()
progression_counts.columns = ["action", "count"]

# Plotly Stacked Bar Chart with Line Plot
fig = px.bar(progression_counts, x="action", y="count", color="action",
             labels={"action": "Action", "count": "Count"},
             title="Progression of Clicks",
             color_discrete_map={"opened": "blue", "meeting_link_click": "orange", "responded": "green"})

# Add line plot
line_fig = px.line(progression_counts, x="action", y="count", labels={"action": "Action", "count": "Count"},
                   line_shape="linear", markers=True, line_dash_sequence=["solid"])

# Add line plot traces to the bar chart
for trace in line_fig.data:
    fig.add_trace(trace)

# Adjust the width
fig.update_layout(width=600)  # Set your desired width

# Use a custom color scale with different shades for each action
fig.update_traces(marker_line_width=0.5, opacity=0.8)

fig.show()

In [41]:
import plotly.express as px

type_counts = df['Type'].value_counts().reset_index()
type_counts.columns = ['Type', 'Count']

# Plotly Horizontal Bar Chart with Gradient Color Scale
fig = px.bar(type_counts, x='Count', y='Type', color='Count',
             labels={'Type': 'Type', 'Count': 'Count'},
             title='Distribution of Types',
             orientation='h',
             color_continuous_scale='Viridis_r',  # You can choose any other color scale
             )

# Rotate x-axis labels
fig.update_layout(xaxis=dict(tickangle=45))

# Adjust the width
fig.update_layout(width=800)  # Set your desired width

fig.show()


# Descriptive Text
descriptive_text = """
The bar chart illustrates the distribution of different types in the dataset. 
'Example 1' is the most prevalent type with 81 occurrences, followed by 'Marketing Analytics Series' with 26 occurrences. 
'Hr Consulting Series' and 'Series 1' also contribute significantly with 18 and 10 occurrences, respectively. 
'Series IT Solutions', 'Legal Services', and 'Financial Advisory Series' are less frequent but still noteworthy with 9, 5, and 5 occurrences, respectively.
"""
print(descriptive_text)


The bar chart illustrates the distribution of different types in the dataset. 
'Example 1' is the most prevalent type with 81 occurrences, followed by 'Marketing Analytics Series' with 26 occurrences. 
'Hr Consulting Series' and 'Series 1' also contribute significantly with 18 and 10 occurrences, respectively. 
'Series IT Solutions', 'Legal Services', and 'Financial Advisory Series' are less frequent but still noteworthy with 9, 5, and 5 occurrences, respectively.



### Insights:

* `Opened`:
Conversion Rate (Opened): The proportion of emails that were opened is 114 out of 154, resulting in a conversion rate of approximately 74.03%. This indicates that a significant portion of recipients engaged with the email by opening it.

* `Meeting Link Click`:
Conversion Rate (Meeting Link Click): Among those who opened the email (114), 46 individuals clicked on the meeting link. The conversion rate for meeting link clicks among those who opened the email is approximately 40.35%. This suggests a positive engagement level, as a substantial portion of those who opened the email proceeded to interact further by clicking on the meeting link.
Responded:

* `Conversion Rate (Responded)`: Out of the total opened emails (114), 8 recipients responded. The conversion rate for responses among those who opened the email is approximately 7.02%. While responding represents a deeper level of engagement than simply opening or clicking, the lower conversion rate indicates that responding is a less common action.

In [43]:
df[df["responded"]==1]

,Type,subject,body,opened,responded,meeting_link_click
10,example 1,Can We Help Optimize Your Marketing Efforts?,"Good day, [Recipient's Name],\n\nAre you harne...",1,1,0
13,example 1,Missed Opportunity? Let's Talk Strategy 📊,"Hey [Recipient's Name],\n\nAre you sure you wa...",1,1,0
34,example 1,Can We Help [Company Name] Excel Today?,"Dear [Recipient's Name],\n\nI'm reaching out a...",1,1,0
58,marketing analytics series,Your Marketing Can Be Smarter 🧠,"Hello [Recipient's Name],\n\nNoticed we missed...",1,1,0
62,series it solutions,Elevate Your IT Game Before It's Too Late...,"Hi again [Recipient's Name],\n\nI noticed my p...",1,1,0
81,example 1,Let's Make Your Data Work for You! 📊🛠,"Hey [Recipient's Name],\n\nIt's [Your Name] ag...",1,1,0
100,series it solutions,Let's Take Your IT to New Heights ✈️ - Last Ch...,"Hey [Recipient's Name],\n\nOversaw my last mes...",1,1,1
137,series 1,Need Clarity in Your Data? Let's Unlock Your M...,"Hello [Recipient's Name],\n\nI noticed my prev...",1,1,0


In [42]:
df.to_excel("db/processed_data.xlsx",index=False)

`Further Analysis`:

* Correlation Analysis: 
Explore the correlation between different features and the target variables. For example, examine whether certain types or subjects are more likely to result in opened emails.

* Time-Based Analysis: If applicable, analyze the dataset over time to identify any temporal patterns or trends. Sending and Receiving time of mails can help us to reach out to them at work timings only thus enhancing click though rates.

* Content Analysis: Conduct a content analysis of the 'subject' and 'body' columns to identify common themes or keywords associated with higher engagement. Word cloud for click and no click will give insights and patterns to focus in future mails.
